# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [2]:
from jorbit.particle import Particle
from jorbit.system import System

obj = Horizons(id="274301", location="@0", epochs=t.tdb.jd)
vecs = obj.vectors(refplane="earth")

x = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

p = Particle(x=x, v=v, time=t, name="wiki")
system = System(particles=[p], acceleration_func="newtonian planets")

print(system.acceleration_func(system.state))

system

[[ 2.61030935e-05 -2.90119924e-05 -8.52860259e-06]]


*************
jorbit System
 time: 2460645.50080073
 particles: [Particle: wiki]
*************

In [3]:
from jorbit.observation import Observations

obj = Horizons(id="274301", location="695", epochs=t.jd)
eph = obj.ephemerides(extra_precision=True, quantities="1")
coord = SkyCoord(eph["RA"][0], eph["DEC"][0], unit=(u.deg, u.deg), frame="icrs")
print(coord)

obs = Observations(
    observed_coordinates=coord,
    times=t,
    observatories="695@399",
    astrometric_uncertainties=1 * u.arcsec,
    verbose=True,
)
obs

<SkyCoord (ICRS): (ra, dec) in deg
    (152.01007338, 5.09857717)>


Observations with 1 set(s) of observations

In [4]:
@jax.jit
def on_sky(
    state: SystemState,
    acc_func,
    observer_position,
):
    a0 = acc_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)

    def scan_func(carry, scan_over):
        xz = carry
        earth_distance = jnp.linalg.norm(xz - observer_position)
        light_travel_time = earth_distance * INV_SPEED_OF_LIGHT

        positions, final_system_state, final_integrator_state = ias15_evolve(
            state,
            acc_func,
            jnp.array([state.time - light_travel_time]),
            initial_integrator_state,
            n_steps=5,
        )

        return final_system_state.positions, None

    xz = jax.lax.scan(scan_func, state.positions, None, length=2)[0]

    X = xz - observer_position
    calc_ra = jnp.mod(jnp.arctan2(X[:, 1], X[:, 0]) + 2 * jnp.pi, 2 * jnp.pi)
    calc_dec = jnp.pi / 2 - jnp.arccos(X[:, -1] / jnp.linalg.norm(X, axis=1))
    return calc_ra, calc_dec


a, b = on_sky(system.state, system.acceleration_func, obs.observer_positions[0])
c = SkyCoord(a, b, unit=(u.rad, u.rad))

c.separation(coord).to(u.microarcsecond)

<Angle [1.1507072] uarcsec>

In [5]:
%%timeit

on_sky(system.state, system.acceleration_func, obs.observer_positions[0])[
    0
].block_until_ready()

541 μs ± 2.56 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
@jax.jit
def tmp(state):
    a0 = system.acceleration_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)
    return initial_integrator_state


%timeit tmp(system.state).g.p0.block_until_ready()

213 μs ± 85.8 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [8]:
obj = Horizons(id="274301", location="@0", epochs=t.jd)

vec_earth = obj.vectors(refplane="earth")
vec_ecliptic = obj.vectors(refplane="ecliptic")

elements_earth = obj.elements(refplane="earth")
elements_ecliptic = obj.elements(refplane="ecliptic")

In [9]:
vec_earth

<Table masked=True length=1>
         targetname          datetime_jd ...       range_rate     
            ---                   d      ...         AU / d       
           str28               float64   ...        float64       
---------------------------- ----------- ... ---------------------
274301 Wikipedia (2008 QH24)   2460645.5 ... 0.0003688611274302079

In [10]:
vec_ecliptic

<Table masked=True length=1>
         targetname          datetime_jd ...       range_rate     
            ---                   d      ...         AU / d       
           str28               float64   ...        float64       
---------------------------- ----------- ... ---------------------
274301 Wikipedia (2008 QH24)   2460645.5 ... 0.0003688611274302083

In [11]:
from jorbit.astrometry.transformations import (
    icrs_to_horizons_ecliptic,
    horizons_ecliptic_to_icrs,
)

v1 = jnp.array([vec_earth["x"][0], vec_earth["y"][0], vec_earth["z"][0]])
v2 = jnp.array([vec_ecliptic["x"][0], vec_ecliptic["y"][0], vec_ecliptic["z"][0]])

icrs_to_horizons_ecliptic(v1) - v2, horizons_ecliptic_to_icrs(v2) - v1

(Array([ 0.00000000e+00,  0.00000000e+00, -5.55111512e-17], dtype=float64),
 Array([0., 0., 0.], dtype=float64))

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [2]:
obj = Horizons(id="274301", location="500@10", epochs=t.jd)
elements_earth_helio = obj.elements(refplane="earth")
elements_ecliptic_helio = obj.elements(refplane="ecliptic")

obj = Horizons(id="274301", location="@0", epochs=t.jd)
elements_earth_bary = obj.elements(refplane="earth")
elements_ecliptic_bary = obj.elements(refplane="ecliptic")


obj = Horizons(id="274301", location="@0", epochs=t.jd)
vec_earth = obj.vectors(refplane="earth")
vec_ecliptic = obj.vectors(refplane="ecliptic")
v1 = jnp.array([vec_earth["x"][0], vec_earth["y"][0], vec_earth["z"][0]])
v2 = jnp.array([vec_ecliptic["x"][0], vec_ecliptic["y"][0], vec_ecliptic["z"][0]])

In [3]:
elements_earth_helio

<Table masked=True length=1>
         targetname          datetime_jd ...         Q                 P        
            ---                   d      ...         AU                d        
           str28               float64   ...      float64           float64     
---------------------------- ----------- ... ----------------- -----------------
274301 Wikipedia (2008 QH24)   2460645.5 ... 2.732190197735032 1341.571548158881

In [4]:
elements_earth_bary

<Table masked=True length=1>
         targetname          datetime_jd ...         Q                 P        
            ---                   d      ...         AU                d        
           str28               float64   ...      float64           float64     
---------------------------- ----------- ... ----------------- -----------------
274301 Wikipedia (2008 QH24)   2460645.5 ... 2.733512339749321 1338.093878421352

In [5]:
elements = {
    "a": jnp.array([elements_earth_bary["a"][0]]),
    "ecc": jnp.array([elements_earth_bary["e"][0]]),
    "inc": jnp.array([elements_earth_bary["incl"][0]]),
    "Omega": jnp.array([elements_earth_bary["Omega"][0]]),
    "omega": jnp.array([elements_earth_bary["w"][0]]),
    "nu": jnp.array([elements_earth_bary["nu"][0]]),
}
elements

from jorbit.astrometry.transformations import elements_to_cartesian

c = elements_to_cartesian(**elements)
c

(Array([[-1.78354686,  1.9728244 ,  0.57967272]], dtype=float64),
 Array([[-0.00752443, -0.00576862, -0.00178665]], dtype=float64))

In [6]:
c[0] - v1

Array([[-8.88178420e-16, -1.99840144e-15, -5.55111512e-16]], dtype=float64)